# Building code for NYSM +/-15 day averages
- https://docs.xarray.dev/en/stable/examples/monthly-means.html
- https://docs.xarray.dev/en/stable/generated/xarray.Dataset.rolling.html#xarray.Dataset.rolling

In [1]:
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime, timedelta
import os

#### Read in NYSM data
- Data is saved locally as csv files for each day with file naming convention "20210815.csv"
    - Each day contains all sites with meteorological data saved out every 5 minutes
- Read in each day's dataframe and concatenate into one large dataframe containing all days/sites

In [2]:
# Set paths and the initial date of NYSM data
# nysm_sites = pd.read_csv("/spare11/atm533/data/nysm_sites.csv")
path = "/kt11/ktyle/mesonet/" # use your path
date_init = datetime.strptime('01-01-17', '%m-%d-%y')
days_from_init = 365*2 #5+257 # Ex: to read in each day's CSV set to 365*5, we have data through Sep 13, 2022

In [3]:
# create list of strings in the form of "yyyymmdd" to be used for reading in csv datasets
def date_string(date_val):
    return f"{'{:04d}'.format(date_val.year)}{'{:02d}'.format(date_val.month)}{'{:02d}'.format(date_val.day)}"

dates = []
for x in range(0, days_from_init): 
    date = date_string(date_init + timedelta(days = x))
    dates.append(date)
    
# create list of all files to be read in using list of dates (as strings) as defined in previous cell
all_files = []
for day in dates:
    all_files.append(f"{path}{day}.csv")

In [4]:
%%time 

# for each csv file in list, read in the data as a dataframe and append dataframes to list "li"
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# concatenate all dataframes in list to get one large dataframe for each day
nysm_data = pd.concat(li, axis=0, ignore_index=True)

CPU times: user 49.4 s, sys: 7.81 s, total: 57.2 s
Wall time: 57.6 s


In [5]:
len(li)

730

In [7]:
# subset dataframe to fields of interest and rename column name

nysm_data = nysm_data[["station", "time", "temp_2m [degC]"]]
nysm_data = nysm_data.rename(columns={"temp_2m [degC]": "temp_2m[C]"})
nysm_data['year'] = nysm_data['time'].str[0:4] # added for averaging over year 

# nysm_data.columns
# nysm_data[["station", "time"]].nunique()
# 125*105120

In [8]:
print(nysm_data.columns)
print(nysm_data[["station", "time"]].nunique())
print(126*599616)
print(nysm_data.shape)

Index(['station', 'time', 'temp_2m[C]', 'year'], dtype='object')
station       126
time       210240
dtype: int64
75551616
(26008992, 4)


In [9]:
nysm_data.shape
nysm_data.head()

,station,time,temp_2m[C],year
0,ADDI,2017-01-01 00:00:00 UTC,-0.8,2017
1,ADDI,2017-01-01 00:05:00 UTC,-0.7,2017
2,ADDI,2017-01-01 00:10:00 UTC,-0.5,2017
3,ADDI,2017-01-01 00:15:00 UTC,-0.4,2017
4,ADDI,2017-01-01 00:20:00 UTC,-0.2,2017


In [10]:
np.mean(nysm_data[nysm_data['station'] == 'ADDI'][0:288]['temp_2m[C]'])

0.12604166666666652

#### Use xarray's .rolling function to calculate the 15-day rolling average from each date
- First convert the large dataframe of mesonet data into xarray dataset with station and time as the coordinates
- Average by date so that there is one value for each site and date (rather than 288, every 5 mins, per day)
- Find each day's 15-day rolling average, centered around the date (i.e. includes the 7 days prior and after)

In [11]:
# %%time

# # set station and time to be indices of the df so that the conversion to xarray dataset makes them coordinates
# nysm_df_work = nysm_data[["station", "time", "temp_2m[C]"]].set_index(["station", "time"]) #, "year"]
# print(nysm_data.shape)
# # convert to dataset
# nysm_ds_work = nysm_df_work.to_xarray()

# # convert time coordinate of the dataset to datetime format
# # NOTE: must be executed twice (last subcomment here by Vinod: https://stackoverflow.com/questions/62572678/xarray-coords-conversion-to-datetime64)
# nysm_ds_work["time"] = pd.DatetimeIndex(nysm_ds_work["time"].values)
# nysm_ds_work["time"] = pd.DatetimeIndex(nysm_ds_work["time"].values)

In [12]:
# nysm_data

In [13]:
%%time

# set station and time to be indices of the df so that the conversion to xarray dataset makes them coordinates
nysm_df = nysm_data[["station", "time", "year", "temp_2m[C]"]].set_index(["station", "year",  "time"]) #, "year"]
print(nysm_data.shape)
# convert to dataset
nysm_ds = nysm_df.to_xarray()

# convert time coordinate of the dataset to datetime format
# NOTE: must be executed twice (last subcomment here by Vinod: https://stackoverflow.com/questions/62572678/xarray-coords-conversion-to-datetime64)
nysm_ds["time"] = pd.DatetimeIndex(nysm_ds["time"].values)
nysm_ds["time"] = pd.DatetimeIndex(nysm_ds["time"].values)

(26008992, 4)
CPU times: user 41.1 s, sys: 1.7 s, total: 42.8 s
Wall time: 43.7 s


In [16]:
nysm_ds

<xarray.Dataset>
Dimensions:     (station: 126, year: 2, time: 210240)
Coordinates:
  * station     (station) object 'ADDI' 'ANDE' 'BATA' ... 'WHIT' 'WOLC' 'YORK'
  * year        (year) object '2017' '2018'
  * time        (time) datetime64[ns] 2017-01-01 ... 2018-12-31T23:55:00
Data variables:
    temp_2m[C]  (station, year, time) float64 -0.8 -0.7 -0.5 ... 5.0 5.1 5.0

In [59]:
# nysm_ds.sel(station = 'ADDI').values

In [15]:
# nysm_ds_work

In [17]:
# nysm_ds.groupby("time.date")

In [18]:
# get average value by day (removing hour/minute variable and leaving just one value per day)
# ds_work = nysm_ds_work.groupby("time.date").mean("time")

In [65]:
# nysm_ds

<xarray.Dataset>
Dimensions:     (station: 126, year: 2, time: 210240)
Coordinates:
  * station     (station) object 'ADDI' 'ANDE' 'BATA' ... 'WHIT' 'WOLC' 'YORK'
  * year        (year) object '2017' '2018'
  * time        (time) datetime64[ns] 2017-01-01 ... 2018-12-31T23:55:00
Data variables:
    temp_2m[C]  (station, year, time) float64 -0.8 -0.7 -0.5 ... 5.0 5.1 5.0

In [19]:
# get average value by day (removing hour/minute variable and leaving just one value per day)
ds = nysm_ds.groupby("time.date").mean("time")

/knight/anaconda_aug22/envs/aug22_env/lib/python3.10/site-packages/flox/aggregate_flox.py:105: RuntimeWarning: invalid value encountered in true_divide
  out /= nanlen(group_idx, array, size=size, axis=axis, fill_value=0)


In [20]:
ds

<xarray.Dataset>
Dimensions:     (station: 126, year: 2, date: 730)
Coordinates:
  * station     (station) object 'ADDI' 'ANDE' 'BATA' ... 'WHIT' 'WOLC' 'YORK'
  * year        (year) object '2017' '2018'
  * date        (date) object 2017-01-01 2017-01-02 ... 2018-12-30 2018-12-31
Data variables:
    temp_2m[C]  (station, year, date) float64 0.126 -1.302 ... -1.98 -0.008333

In [39]:
# selecting just 2017, will either have values for 2017 data or nans otherwise
# ds.sel(station = 'ADDI', year = '2017')['temp_2m[C]'][0:365]
# ds.sel(station = 'ADDI', year = '2017')['temp_2m[C]'][365:720]

In [40]:
# ds_work["date"] = pd.DatetimeIndex(ds_work["date"].values)
# ds_work["date"] = pd.DatetimeIndex(ds_work["date"].values)

# ds_work

In [42]:
ds["date"] = pd.DatetimeIndex(ds["date"].values)
ds["date"] = pd.DatetimeIndex(ds["date"].values)

ds

<xarray.Dataset>
Dimensions:     (station: 126, year: 2, date: 730)
Coordinates:
  * station     (station) object 'ADDI' 'ANDE' 'BATA' ... 'WHIT' 'WOLC' 'YORK'
  * year        (year) object '2017' '2018'
  * date        (date) datetime64[ns] 2017-01-01 2017-01-02 ... 2018-12-31
Data variables:
    temp_2m[C]  (station, year, date) float64 0.126 -1.302 ... -1.98 -0.008333

In [43]:
# %%time

# # By date, calculate rolling average of past +/- 5 days. Adjust this to +/- 15 once I get all data
# # Rolling window is set to be 5 dates (dates are days)
# # Note that there is a parameter, min_periods, whose default is set to None which equates to it being equal the size of the window. In other words, the if the rolling window is 5 and centered, that means we need 2 before, term in middle, 2 terms after, so we would have NANs for the first two terms
# # Center = True, as it suggests, means that at a date coordinate (a date row) it is at the center of the rolling average
# # Can confirm this is working as we think by changing it to center = True and seeing that the first 2 are nans

# rolling_5day_avg_work = ds_work.rolling({"date":7}, center = True).mean()

# # add the rolling values to the main dataset 
# ds_work['rolling_temp_2m[C]'] = rolling_5day_avg_work['temp_2m[C]']
# ds_work

In [44]:
%%time

# By date, calculate rolling average of past +/- 5 days. Adjust this to +/- 15 once I get all data
# Rolling window is set to be 5 dates (dates are days)
# Note that there is a parameter, min_periods, whose default is set to None which equates to it being equal the size of the window. In other words, the if the rolling window is 5 and centered, that means we need 2 before, term in middle, 2 terms after, so we would have NANs for the first two terms
# Center = True, as it suggests, means that at a date coordinate (a date row) it is at the center of the rolling average
# Can confirm this is working as we think by changing it to center = True and seeing that the first 2 are nans

rolling_5day_avg = ds.rolling({"date":7}, center = True).mean()

# add the rolling values to the main dataset 
ds['rolling_temp_2m[C]'] = rolling_5day_avg['temp_2m[C]']
ds

CPU times: user 7.24 ms, sys: 1.17 ms, total: 8.42 ms
Wall time: 8.15 ms


<xarray.Dataset>
Dimensions:             (station: 126, year: 2, date: 730)
Coordinates:
  * station             (station) object 'ADDI' 'ANDE' 'BATA' ... 'WOLC' 'YORK'
  * year                (year) object '2017' '2018'
  * date                (date) datetime64[ns] 2017-01-01 ... 2018-12-31
Data variables:
    temp_2m[C]          (station, year, date) float64 0.126 -1.302 ... -0.008333
    rolling_temp_2m[C]  (station, year, date) float64 nan nan nan ... nan nan

In [45]:
# len(ds['date'])

# ds['date'][200]

In [43]:
# print(np.mean(ds_work.sel(station = 'QUEE')['temp_2m[C]'].values[112:119]))
# print(ds_work.sel(station = 'QUEE')['rolling_temp_2m[C]'].values[115])

14.647569444444445
14.647569444444443


In [46]:
# spot check a few values (when rolling average was 7, 3 on each side)

print(np.mean(ds.sel(station = 'QUEE', year = '2017')['temp_2m[C]'].values[112:119]))
print(ds.sel(station = 'QUEE', year = '2017')['rolling_temp_2m[C]'].values[115])

14.647569444444445
14.647569444444443


In [71]:
ds

<xarray.Dataset>
Dimensions:             (station: 126, year: 2, date: 730)
Coordinates:
  * station             (station) object 'ADDI' 'ANDE' 'BATA' ... 'WOLC' 'YORK'
  * year                (year) object '2017' '2018'
  * date                (date) datetime64[ns] 2017-01-01 ... 2018-12-31
Data variables:
    temp_2m[C]          (station, year, date) float64 0.126 -1.302 ... -0.008333
    rolling_temp_2m[C]  (station, year, date) float64 nan nan nan ... nan nan

#### Average each date's rolling 15-day anomaly to get the DAY anomaly
- Using April 15th as an example: in the code above, we calculated the rolling 15-day average for April 15th as the avg temp from April 8 through April 22, but this was done BY YEAR. But we want an "April 15th" average from over the years to calculuate the anomaly from, so we will average the April 15th "rolling_temp" across years to get one average temp value for April 15th, which will then be used to subtract from each dates (y/m/d) to get the anomaly for each day. 
    - Note that calculating the 15-day rolling mean for each year and then averaging all year's averages together is equivalent to averaging all at once

In [108]:
df = ds.to_dataframe()

In [112]:
df.reset_index(inplace=True)

In [113]:
df['monthday'] = df['date'].astype(str).str[5:10]

In [114]:
df

,station,year,date,temp_2m[C],rolling_temp_2m[C],monthday
0,ADDI,2017,2017-01-01,0.126042,NaN,01-01
1,ADDI,2017,2017-01-02,-1.302431,NaN,01-02
2,ADDI,2017,2017-01-03,2.219097,NaN,01-03
3,ADDI,2017,2017-01-04,1.353472,-3.975397,01-04
4,ADDI,2017,2017-01-05,-7.119792,-5.748710,01-05
...,...,...,...,...,...,...
183955,YORK,2018,2018-12-27,-0.560069,1.237153,12-27
183956,YORK,2018,2018-12-28,8.614583,1.135317,12-28
183957,YORK,2018,2018-12-29,2.659028,NaN,12-29
183958,YORK,2018,2018-12-30,-1.980208,NaN,12-30


In [115]:
df_sub = df[['station', 'monthday', 'rolling_temp_2m[C]']]

In [161]:
df_anom_monthday = df_sub.groupby(['monthday', 'station']).mean()
df_anom_monthday.reset_index(inplace=True)
df_anom_monthday.rename(columns = {'rolling_temp_2m[C]':'rel_day_temp_2m[C]'}, inplace = True)
df_anom_monthday

,monthday,station,rel_day_temp_2m[C]
0,01-01,ADDI,NaN
1,01-01,ANDE,NaN
2,01-01,BATA,NaN
3,01-01,BEAC,NaN
4,01-01,BELD,NaN
...,...,...,...
45985,12-31,WFMB,NaN
45986,12-31,WGAT,NaN
45987,12-31,WHIT,NaN
45988,12-31,WOLC,NaN


In [162]:
df_count_monthday = df_sub.groupby(['monthday', 'station']).count()
df_count_monthday.reset_index(inplace=True)
df_count_monthday.rename(columns = {'rolling_temp_2m[C]':'count_day_temp_2m[C]'}, inplace = True)
df_count_monthday

,monthday,station,count_day_temp_2m[C]
0,01-01,ADDI,0
1,01-01,ANDE,0
2,01-01,BATA,0
3,01-01,BEAC,0
4,01-01,BELD,0
...,...,...,...
45985,12-31,WFMB,0
45986,12-31,WGAT,0
45987,12-31,WHIT,0
45988,12-31,WOLC,0


In [140]:
df_anom_monthday[(df_anom_monthday['station'] =='RAQU') & (df_anom_monthday['monthday'] == '01-24')]

,monthday,station,rolling_temp_2m[C]
2984,01-24,RAQU,-2.449826


In [167]:
df_merge = df[['monthday', 'station', 'temp_2m[C]']].merge(df_anom_monthday, how='inner', left_on = ['monthday', 'station'], right_on=['monthday', 'station'])
df_final= df_merge.merge(df_count_monthday, how='inner', left_on = ['monthday', 'station'], right_on=['monthday', 'station'])

In [168]:
df_final[16000:16007]

,monthday,station,temp_2m[C],rel_day_temp_2m[C],count_day_temp_2m[C]
16000,12-17,BRAN,-5.001736,0.053026,2
16001,12-17,BRAN,NaN,0.053026,2
16002,12-17,BRAN,NaN,0.053026,2
16003,12-17,BRAN,2.331944,0.053026,2
16004,12-18,BRAN,1.831597,0.594929,2
16005,12-18,BRAN,NaN,0.594929,2
16006,12-18,BRAN,NaN,0.594929,2


In [170]:
df_final['anomaly_temp_2m[C]'] = df_final['temp_2m[C]'] - df_final['rel_day_temp_2m[C]']

In [176]:
df_final[170000:170007]

,monthday,station,temp_2m[C],rel_day_temp_2m[C],count_day_temp_2m[C],anomaly_temp_2m[C]
170000,06-10,WARW,18.945833,18.635803,2,0.310030
170001,06-10,WARW,NaN,18.635803,2,NaN
170002,06-10,WARW,NaN,18.635803,2,NaN
170003,06-10,WARW,16.513889,18.635803,2,-2.121914
170004,06-11,WARW,24.382639,19.676602,2,4.706037
170005,06-11,WARW,NaN,19.676602,2,NaN
170006,06-11,WARW,NaN,19.676602,2,NaN


In [ ]:
# GO BACK THROUGH AND IGNORE THE DAYS WHERE THERE ARE NULLS - many nulls in between days of no nulls

## REFERENCE CODE ONLY
#### Ignore..


In [109]:
# call on a coordinate
nysm_ds.station

# call on a varibale
nysm_ds['temp_2m [degC]']

<xarray.DataArray 'temp_2m [degC]' (station: 126, time: 8929)>
array([[23.3, 22.7, 22.5, ..., 18.2, 18.2, 18.2],
       [21.3, 20.4, 19.7, ..., 16.8, 16.5, 16.4],
       [24.1, 24. , 24. , ..., 22.6, 22.6, 22.5],
       ...,
       [24.7, 24.2, 23.5, ..., 14.3, 14.3, 14.4],
       [23.8, 22.9, 22.5, ..., 22.1, 21.8, 21.8],
       [22.8, 22.2, 21.8, ..., 22.9, 22.8, 22.6]])
Coordinates:
  * station  (station) object 'ADDI' 'ANDE' 'BATA' ... 'WHIT' 'WOLC' 'YORK'
  * time     (time) datetime64[ns] 2020-07-31T23:50:00 ... 2020-08-31T23:55:00

In [ ]:
nysm_ds['temp_2m [degC]'].shape

In [29]:
# REFERENCE ONLY
# "data array" . mean ("dimension to average across"... eliminating that dim completely, it's averaged out if left blank, will average over everything to give ONE mean value)
# produces another array whose length depends on what you averaged over!

# example 1
avg_temp_by_day = nysm_ds['temp_2m [degC]'].mean('station')
print(avg_temp_by_day.shape)

# example 2
avg_temp_by_station = nysm_ds['temp_2m [degC]'].mean('time')
print(avg_temp_by_station.shape)

(8929,)
(126,)


In [39]:
# this is just like SQL - using all data, but group certain rows together based on common (i.e. season) and then still want to average over the time dimension so that dim is gone and we're left with station only
test1 = nysm_ds.groupby("time.season").mean("time")

/knight/anaconda_aug22/envs/aug22_env/lib/python3.10/site-packages/flox/aggregate_flox.py:105: RuntimeWarning: invalid value encountered in true_divide
  out /= nanlen(group_idx, array, size=size, axis=axis, fill_value=0)


In [43]:
test1

<xarray.Dataset>
Dimensions:                                (station: 126, date: 32)
Coordinates:
  * station                                (station) object 'ADDI' ... 'YORK'
  * date                                   (date) object 2020-07-31 ... 2020-...
Data variables: (12/28)
    temp_2m [degC]                         (station, date) float64 23.3 ... 1...
    temp_9m [degC]                         (station, date) float64 23.5 ... 1...
    relative_humidity [percent]            (station, date) float64 57.1 ... 80.6
    precip_incremental [mm]                (station, date) float64 0.0 ... 0.0
    precip_local [mm]                      (station, date) float64 0.0 ... 0....
    precip_max_intensity [mm/min]          (station, date) float64 0.0 ... 0.0
    ...                                     ...
    soil_temp_05cm [degC]                  (station, date) float64 22.1 ... 1...
    soil_temp_25cm [degC]                  (station, date) float64 20.1 ... 2...
    soil_temp_50cm [degC]                  (station, date) float64 19.1 ... 2...
    soil_moisture_05cm [m^3/m^3]           (station, date) float64 0.08 ... 0...
    soil_moisture_25cm [m^3/m^3]           (station, date) float64 0.2 ... 0....
    soil_moisture_50cm [m^3/m^3]           (station, date) float64 0.27 ... 0.21

In [38]:
test2= nysm_ds.groupby("time.season")
type(test2)

xarray.core.groupby.DatasetGroupBy

In [10]:
ds_sub = nysm_ds.sel(time=nysm_ds.time.dt.month.isin([4]))

In [13]:
nysm_ds.time.dt.date

<xarray.DataArray 'date' (time: 8929)>
array([datetime.date(2020, 7, 31), datetime.date(2020, 8, 1),
       datetime.date(2020, 8, 1), ..., datetime.date(2020, 8, 31),
       datetime.date(2020, 8, 31), datetime.date(2020, 8, 31)],
      dtype=object)
Coordinates:
  * time     (time) datetime64[ns] 2020-07-31T23:50:00 ... 2020-08-31T23:55:00

In [99]:
# REFERENCE -- Check that this rolling average is working correctly, not getting mixed up by sites or anything

# SUBSET DATASET: https://stackoverflow.com/questions/38846323/python-xarray-dataset-sel-select-multiple-values-along-one-dimension
# way 1
y = ds_avg_by_site_date.where(ds_avg_by_site_date.station=='WOLC', drop = True)#, "date" :'2020-08-06'})
y

# way 2 (using sel and also datetime issue)
# Using .sel (datetime format issue..)
# g = ds_avg_by_site_date.sel(ds_avg_by_site_date.date == dt.date(2020, 7, 31)) # DATETIME ISSUE
# g

# print out the temps for WOLC then avg in cell below to make sure it works
y["temp_2m [degC]"]

<xarray.DataArray 'temp_2m [degC]' (station: 1, date: 32)>
array([[23.8       , 22.29583333, 25.55173611, 23.64409722, 20.59097222,
        20.34097222, 18.16458333, 19.39201389, 21.21284722, 22.58715278,
        25.67291667, 26.40486111, 22.86041667, 21.67986111, 22.54895833,
        22.99444444, 22.41805556, 20.709375  , 19.25034722, 18.06284722,
        18.01909722, 20.62152778, 22.36423611, 23.75208333, 24.45972222,
        23.28298611, 18.26666667, 20.33854167, 20.16666667, 21.65798611,
        18.71840278, 17.40381944]])
Coordinates:
  * station  (station) object 'WOLC'
  * date     (date) object 2020-07-31 2020-08-01 ... 2020-08-30 2020-08-31

In [8]:
num = 23.8 +22.29583333+25.55173611+23.64409722+20.59097222
num/5

23.176527776

In [9]:
# check that this was the first non nan value for WOLC
# rolling_5day_avg
rolling_5day_avg.where(rolling_5day_avg.station=='WOLC', drop = True) # confirmed! the first value is 23.18

<xarray.Dataset>
Dimensions:                                (station: 1, date: 32)
Coordinates:
  * station                                (station) object 'WOLC'
  * date                                   (date) object 2020-07-31 ... 2020-...
Data variables: (12/28)
    temp_2m [degC]                         (station, date) float64 nan ... nan
    temp_9m [degC]                         (station, date) float64 nan ... nan
    relative_humidity [percent]            (station, date) float64 nan ... nan
    precip_incremental [mm]                (station, date) float64 nan ... nan
    precip_local [mm]                      (station, date) float64 nan ... nan
    precip_max_intensity [mm/min]          (station, date) float64 nan ... nan
    ...                                     ...
    soil_temp_05cm [degC]                  (station, date) float64 nan ... nan
    soil_temp_25cm [degC]                  (station, date) float64 nan ... nan
    soil_temp_50cm [degC]                  (station, date) float64 nan ... nan
    soil_moisture_05cm [m^3/m^3]           (station, date) float64 nan ... nan
    soil_moisture_25cm [m^3/m^3]           (station, date) float64 nan ... nan
    soil_moisture_50cm [m^3/m^3]           (station, date) float64 nan ... nan

#### Reference only 11/27: manually creating rolling dates read in

In [ ]:
# method 1
datetime_mmyy = datetime.strptime('02-04', '%m-%d')
datetime_mmyy

In [ ]:
# back to method 1
def date_string(date_val):
    return f"{'{:02d}'.format(date_val.month)}{'{:02d}'.format(date_val.day)}"


def read_csv_rolling(centerdate, numdays): # change numdays to rolling_half
    dateList = []
    for x in range (0, numdays+1):
        if x == 0:
            center_date = date_string(centerdate)
            dateList.append(center_date)
        else:
            prior_date = date_string(centerdate - timedelta(days = x))
            dateList.append(prior_date)
            next_date = date_string(centerdate + timedelta(days = x))
            dateList.append(next_date)
    return dateList

day_range_read = read_csv_rolling(datetime_mmyy, 7)

print(day_range_read)
print(len(day_range_read))


In [ ]:
%%time 

li = []

for day in day_range_read:
    # list files for a given day, e.g. if day is 0204, this will list mesonet csv data for 20160204, 20170204, ... 20220204
    all_files = glob(f"{path}*{day}.csv")

    # for each file (year) in list, read in the data csv data and append it to outer list
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

nysm_data = pd.concat(li, axis=0, ignore_index=True)


In [ ]:
# method 1
# set station and time to be indices of the df so that the conversion to xarray dataset makes them coordinates
nysm_data = nysm_data.set_index(["station", "time"])

# convert to dataset
nysm_ds = nysm_data.to_xarray()

# convert time coordinate of the dataset to datetime format
# NOTE: must be executed twice (last subcomment here by Vinod: https://stackoverflow.com/questions/62572678/xarray-coords-conversion-to-datetime64)
nysm_ds["time"] = pd.DatetimeIndex(nysm_ds["time"].values)
nysm_ds["time"] = pd.DatetimeIndex(nysm_ds["time"].values)

nysm_ds

In [203]:
# issue subsetting slicing by dates

ds_avg_by_site_date.sel(station = 'BATA', date in (datetime.date(2017, 1, 1), datetime.date(2017, 1, 3)))#['temp_2m[C]'].values

SyntaxError: positional argument follows keyword argument (3043326369.py, line 3)

In [ ]:
import xarray as xr
import numpy as np
data = xr.DataArray([1, 2, 3], dims='x', coords={'x': [10, 20, 30]})
# data
data_newcoord = data.assign_coords(y='newvaluee')
# data
data_expanded = data_newcoord.expand_dims('y')
data_expanded
# print(data_expanded)

In [ ]:
nysm_ds["time.month"]

In [ ]:
m = ds['date.month']
d = ds['date.day']
t = ds['date.date']


# day_ = m+d
# day_

mo_day = []
for i in range(0, len(ds['date'])):
    day = f"{'{:02d}'.format(ds['date.month'][i].values.item())}{'{:02d}'.format(ds['date.day'][i].values.item())}"
    mo_day.append(day)


In [ ]:
ds['yeartry'] = ds["date.year"]
ds['monttry'] = ds["date.month"]
ds['daytry'] = ds["date.day"]

ds2 = ds.assign_coords(coor_yr=ds["date.year"],coor_mo=ds["date.month"],coor_day=ds["date.day"])



In [ ]:
ds2.swap_dims({'date':'yeartry' }).drop('date')

In [ ]:
import xarray as xr

data_newcoord = data.assign_coords(y='coord_value')
data_expanded = data_newcoord.expand_dims('y')
print(data_expanded)

dsavg2 = ds.mean(dim="date.year", skipna=True)

In [53]:
ds

<xarray.Dataset>
Dimensions:             (station: 126, year: 2, date: 730)
Coordinates:
  * station             (station) object 'ADDI' 'ANDE' 'BATA' ... 'WOLC' 'YORK'
  * year                (year) object '2017' '2018'
  * date                (date) datetime64[ns] 2017-01-01 ... 2018-12-31
Data variables:
    temp_2m[C]          (station, year, date) float64 0.126 -1.302 ... -0.008333
    rolling_temp_2m[C]  (station, year, date) float64 nan nan nan ... nan nan

In [60]:
ds["date.month"]

<xarray.DataArray 'month' (date: 730)>
array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12,
...
        1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12])
Coordinates:
  * date     (date) datetime64[ns] 2017-01-01 2017-01-02 ... 2018-12-31

In [54]:
ds_new = ds


In [ ]:
ds_new.assign_coords(mo_day=(((ds.date + 180) % 360) - 180))

#### Ideas for averageing over hyear which ended up doing in df
#### split the date which is currently year_month_day into 1)year and 2) month_day 
    - then I can average over year
#### OR if there is a way to "call on" the month_day from the date coordinate (like we did in group by functions above) we can use the groupby way